# Data Augmentation

Альбументация данных - это такой подход, который используется для расширения обучающей выборки за счёт небольших изменений исходных данных. Например, сдвига данных, отражения по вертикальной/горизонтальной оси, изменение насыщенности пикселей (в случае картинок). 

Помимо увеличения количества данных в случае, когда их исходно мало, альбументация данных помогает избежать переобучения модели, а также помогает увелиить устойчивость работы модели (так как при обучении она видит разные модификации данных). 

Альбументацию можно применять к разным типам данных: 
- аудио
- текст
- изображения
  

Для альбументации изображений можно использовать:
- геометрические преобразования - рандомная обрезка, поворот, отраждение и т.д. 
- изменение цветового пространства - изменение интенсивности, яркости пикселей и т.д.
- фильтрация - размытие, изменение резкости и т.д.
- рандомная зачистка - удаление части исходной картинки 

Для текста можно использовать: 
- перемешивание слов/предложений 
- перестановка слов - замена слов синонимами 
- манипуляция с текстом - перефразирование предложений

Для аудио можно использовать: 
- добавление шумов 
- смещение каналов 
- изменение скорости 

Это только малая часть альбументаций, которую можно использовать. На самом деле их существует намного и намного больше. 


In [ ]:
%load_ext autoreload
%autoreload 2

import albumentations as albu
import cv2
import torchvision
from torchvision.transforms import transforms
import matplotlib.pyplot as plt 
import numpy as np 
from ipywidgets import interact, IntSlider

RANDOM_SEED = 42

In [ ]:
src_data = torchvision.datasets.MNIST(
    "data", train=True, transform=None, download=True
)

In [ ]:
data=list(src_data)

In [ ]:
images, labels = [], []
for info in data:
    img, label = info 
    images.append(np.array(img))
    labels.append(label)

print(f"Number of images: {len(images)}")
print(f"Number of labels: {len(labels)}")

In [ ]:
plt.figure(figsize=(10,5))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(images[i], cmap='gray')
plt.show()

In [ ]:
transform = albu.ShiftScaleRotate(
    shift_limit=0.3,
    scale_limit=0,
    rotate_limit=0,
    interpolation=3,
    border_mode=cv2.BORDER_CONSTANT,
    p=0.9,
    value=255,  # white background for better representation
)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img = transform(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[1].imshow(transformed_img, cmap="gray")

    plt.show()

In [ ]:
transform = albu.GaussianBlur(blur_limit=3, p=0.9)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img = transform(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[1].imshow(transformed_img, cmap="gray")

    plt.show()

In [ ]:
transform = albu.HorizontalFlip(p=0.9)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img = transform(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[1].imshow(transformed_img, cmap="gray")

    plt.show()

In [ ]:
transform = albu.InvertImg(p=0.9)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img = transform(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[1].imshow(transformed_img, cmap="gray")

    plt.show()

In [ ]:
transform = albu.Compose(
    [
        albu.OneOf(
            [
                albu.ShiftScaleRotate(
                    shift_limit=0.5,
                    scale_limit=0,
                    rotate_limit=0,
                    interpolation=3,
                    border_mode=cv2.BORDER_CONSTANT,
                    p=0.9,
                    value=255,  # white background for better representation
                ),
                albu.ShiftScaleRotate(
                    shift_limit=0,
                    scale_limit=0.5,
                    rotate_limit=0,
                    interpolation=3,
                    border_mode=cv2.BORDER_CONSTANT,
                    p=0.9,
                    value=255,  # white background for better representation
                ),
                albu.ShiftScaleRotate(
                    shift_limit=0,
                    scale_limit=0,
                    rotate_limit=50,
                    interpolation=3,
                    border_mode=cv2.BORDER_CONSTANT,
                    p=0.9,
                    value=255,  # white background for better representation
                ),
                albu.InvertImg(p=0.9)
            ]
        )
    ], 
    p=1, 
)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img = transform(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[1].imshow(transformed_img, cmap="gray")

    plt.show()

In [ ]:
class AlbuAugmentationMultichannel:
    def __init__(self, n_images: int):
        ssr_params = dict(
            shift_limit=0.1,
            scale_limit=0.1,
            rotate_limit=10,
            interpolation=3,
            border_mode=cv2.BORDER_CONSTANT,
            p=0.5,
        )

        self.description = [
            albu.OneOf(
                [
                    albu.GaussNoise(p=0.5),
                    albu.MultiplicativeNoise(per_channel=True, p=0.3),
                ],
                p=0.4,
            ),
            albu.OneOf(
                [
                    albu.MotionBlur(blur_limit=3, p=0.2),
                    albu.MedianBlur(blur_limit=3, p=0.2),
                    albu.GaussianBlur(blur_limit=3, p=0.2),
                    albu.Blur(blur_limit=3, p=0.2),
                ],
                p=0.2,
            ),
            albu.OneOf(
                [
                    albu.CLAHE(),
                    albu.Sharpen(),
                    albu.RandomBrightnessContrast(),
                ],
                p=0.3,
            ),
            albu.ShiftScaleRotate(**ssr_params, value=(0,)),
        ]
        images = {"{}{}".format("image", i): "image" for i in range(n_images - 1)}
        self.compose = albu.Compose(self.description, p=1, additional_targets=images)

    def __call__(self, img: list) -> list:
        images = {}
        for i, image in enumerate(img):
            imname = "{}{}".format("image", i - 1) if i > 0 else "image"
            images[imname] = image
        transformed = self.compose(**images)
        img = list(transformed.values())
        return img

## Полезные ссылки

* [Albumentations for image classififcation](https://albumentations.ai/docs/getting_started/image_augmentation/)
* [List of albumentations](https://albumentations.ai/docs/getting_started/transforms_and_targets/)